## Exploring our parsed StackOverflow post / tag data (in a DuckDB table) and running SQL queries on it

In [1]:
from pathlib import Path

import duckdb
import pandas as pd

from ragoverflow_shared.models import PostType

# Set the display option to show decimal numbers in  a more human readable way (no scientific notation)
pd.set_option("display.float_format", "{:,.5f}".format)

In [2]:
# Create a connection to our persistent DB file
conn = duckdb.connect(
    Path.cwd() / "../data/duckdb/stackoverflow_analysis.db"
)

In [3]:
# Show the tabs that we have available
conn.sql("""
SHOW TABLES
""")

┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ stackoverflow_posts │
│ stackoverflow_tags  │
└─────────────────────┘

In [4]:
# Get summary statistics on post net votes
conn.sql(
    """
    SELECT net_votes
    FROM stackoverflow_posts
    """
).df().describe()

,net_votes
count,"275,000.00000"
mean,201.18478
std,368.89065
min,65.00000
25%,82.00000
50%,115.00000
75%,194.00000
max,"34,970.00000"


In [5]:
# Find the top 15 most popular tags
conn.sql(
    """
    SELECT *
    FROM stackoverflow_tags
    ORDER BY count DESC LIMIT 15
    """
)

┌────────────┬─────────┐
│  tag_name  │  count  │
│  varchar   │  int32  │
├────────────┼─────────┤
│ javascript │ 2528894 │
│ python     │ 2192438 │
│ java       │ 1917340 │
│ c#         │ 1615192 │
│ php        │ 1464496 │
│ android    │ 1417189 │
│ html       │ 1187348 │
│ jquery     │ 1034760 │
│ c++        │  806743 │
│ css        │  804268 │
│ ios        │  687246 │
│ sql        │  670860 │
│ mysql      │  661984 │
│ r          │  505710 │
│ reactjs    │  476740 │
├────────────┴─────────┤
│ 15 rows    2 columns │
└──────────────────────┘

In [6]:
# Explore the different post types we have
conn.sql(f"""
    SELECT post_type, COUNT(*) as count
    FROM stackoverflow_posts
    GROUP BY post_type
""").df()

,post_type,count
0,1,93451
1,2,181549


In [7]:
# Explore the top 15 most popular posts
conn.sql(f"""
    SELECT *
    FROM stackoverflow_posts
    ORDER BY net_votes DESC LIMIT 15
""").df()

,post_id,post_type,title,body,tags,net_votes,parent_id
0,11227902,2,,"<p><strong>You are a victim of <a href=""https:...",,34970,11227809
1,927386,2,,<h2>Undo a commit &amp; redo</h2>\n<pre class=...,,28828,927358
2,11227809,1,Why is processing a sorted array faster than p...,"<p>In this C++ code, sorting the data (<em>bef...",|java|c++|performance|cpu-architecture|branch-...,27235,<NA>
3,927358,1,How do I undo the most recent local commits in...,<p>I accidentally committed the wrong files to...,|git|version-control|git-commit|undo|,26311,<NA>
4,2003515,2,,<h1>Executive Summary</h1>\n<pre><code>git pus...,,25818,2003505
5,2003505,1,How do I delete a Git branch locally and remot...,<p>Failed Attempts to Delete a Remote Branch:<...,|git|version-control|git-branch|git-push|git-r...,20387,<NA>
6,6591218,2,,<p>To rename the current branch:</p>\n<pre><co...,,18910,6591213
7,179147,2,,<h1>Amending the most recent commit message</h...,,18566,179123
8,231855,2,,"<p>To understand what <a href=""https://docs.py...",,17987,231767
9,5767357,2,,<p>Find the <code>index</code> of the array el...,,16620,5767325


## Analyze the number of tokens in our posts. This is necessary since our embedding model (which takes in text and outputs embeddings) has a limit on the amount of tokens that the input text can consist of 

In [8]:
from transformers import AutoTokenizer
from bs4 import MarkupResemblesLocatorWarning
import warnings
from ragoverflow_shared.html_cleaner import clean_html

# suppress the beautifulsoup warning about the text content - this warning pops up if the textual content
# we are parsing resembles a filename / pathS since we are only parsing textual content from in-memory data structures,
# the warning can be suppressed
warnings.filterwarnings("ignore", category=MarkupResemblesLocatorWarning)

# Load a pre-trained tokenizer model
# Use a tokenizer instead of a full-fledged embedding generator model
print("Loading tokenizer...")

# BGE model name = "BAAI General Embedding"
# "General" = model is designed to work across many domains (not just code, not just medical text, etc.)
# "Embedding" = the model's purpose is to convert text to dense vectors that capture that text's semantic meaning
tokenizer = AutoTokenizer.from_pretrained("BAAI/bge-small-en-v1.5")


# counts the number of tokens in the given series of text
def count_tokens(text):
    return len(tokenizer.encode(text, add_special_tokens=True))


# Sample 100k posts from our DB
print("Loading posts...")
posts = conn.execute(
    """
    SELECT post_type, title, body
    FROM stackoverflow_posts LIMIT 100_000
    """
).fetchdf()

# Count the number of tokens in each post's content, where content is the HTML-cleaned title + body
print("Counting tokens...")
posts['text'] = posts.apply(
    lambda row: clean_html(row['title']) + '\n\n' + clean_html(row['body'])
    if row['post_type'] == PostType.QUESTION.value
    else clean_html(row['body']),
    axis=1
)
posts['token_count'] = posts['text'].apply(count_tokens)

# Print summary statistics on the token count, so we
print(f"\n{posts['token_count'].describe()}")

/Users/donovanmurray/Code/RagProgrammingAssistant/venv/lib/python3.14/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loading tokenizer...
Loading posts...
Counting tokens...


Token indices sequence length is longer than the specified maximum sequence length for this model (1036 > 512). Running this sequence through the model will result in indexing errors



count   100,000.00000
mean        205.23811
std         323.05861
min           2.00000
25%          70.00000
50%         125.00000
75%         229.00000
max      17,232.00000
Name: token_count, dtype: float64


In [9]:
# since the max token input size for our embeddings model is 512 tokens, find the percent of posts that require embeddings
posts_over_limit = (posts['token_count'] > 512).sum()
percent_over = (posts_over_limit / 100_000) * 100

print(f"{posts_over_limit:,} posts ({percent_over:.1f}%) exceed 512 tokens")
print(f"These will need chunking or will lose information if truncated")

extreme_posts = (posts['token_count'] > 2000).sum()
print(f"{extreme_posts:,} posts exceed 2000 tokens (need 4+ chunks)")

6,906 posts (6.9%) exceed 512 tokens
These will need chunking or will lose information if truncated
431 posts exceed 2000 tokens (need 4+ chunks)


## design decision: since 93% of posts will only need one embedding, for now, we will only generate one embedding per post.
- For the posts that are <= 512 tokens, they will get embedded and uploaded directly
- For the posts that > 512 tokens, we will truncate them, and discard all tokens after the first 512

## future optimization: in the future, posts that have 512+ tokens will be broken into chunks of 512 tokens,
- each chunk will be uploaded one-by-one
- Each chunk can be retrieved independently


## Experiment with our transformer model : do similar texts have similar embeddings?

In [10]:
from sentence_transformers import SentenceTransformer
import numpy as np

# Load the model
model = SentenceTransformer("BAAI/bge-small-en-v1.5")

base_text = "How do I sort a list in Python?"
similar_text = "How to sort a Python array"
different_text = "What is machine learning?"

# Generate embeddings for each of our test texts
test_texts = [base_text, similar_text, different_text]
test_embeddings = model.encode(test_texts)


def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))


# compute the cosine similarity b/w the base text embeddings and our two test embeddings
base_vs_similar = cosine_similarity(test_embeddings[0], test_embeddings[1])
base_vs_different = cosine_similarity(test_embeddings[0], test_embeddings[2])

print(f"Similarity (base vs similar): {base_vs_similar:.4f}")
print(f"Similarity (base vs different): {base_vs_different:.4f}")

Similarity (base vs similar): 0.9116
Similarity (base vs different): 0.4270


In [11]:
queries = [
    "How do I sort a list in Python?",
    "Python list sorting methods",  # Very similar
    "Sort array in JavaScript",  # Same task, different language
    "Python for loops",  # Same language, different topic
    "Machine learning algorithms",  # Completely different
]

embeddings = model.encode(queries)
base = embeddings[0]

for i, query in enumerate(queries[1:], 1):
    sim = cosine_similarity(base, embeddings[i])
    print(f"{sim:.4f} | {query}")

0.8840 | Python list sorting methods
0.7972 | Sort array in JavaScript
0.7120 | Python for loops
0.5210 | Machine learning algorithms
